In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime as dt
pd.set_option('future.no_silent_downcasting', True)

# Get WACC

In [308]:
def get_risk_free_rate():
    """
    Get the current 10-year Treasury yield (risk-free rate) from an API.
    """
    try:
        tnx = yf.Ticker("^TNX")
        current_yield = tnx.history(period="1d")['Close'].iloc[-1] * 0.01
        return round(current_yield, 3)
    except:
        return 0.04
        
def get_wacc(ticker):
    stock= yf.Ticker(ticker)
    #Get Market Cap
    market_cap = stock.info.get('marketCap', 0)

    #Get Total Debt
    total_debt = stock.balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in stock.balance_sheet else 0

    # Get Beta
    beta = stock.info.get('beta', 1)

    # Get Interest Expense & Total Revenue to estimate Cost of Debt (Rd)
    interest_expense = stock.financials.loc['Interest Expense'].iloc[0] if 'Interest Expense' in stock.financials else 0
    cost_of_debt = abs(interest_expense / total_debt) if total_debt else 0.05  # Default 5% if no data

    # Get Tax Rate
    income_tax = stock.financials.loc['Income Tax Expense'].iloc[0] if 'Income Tax Expense' in stock.financials else 0
    ebit = stock.financials.loc['EBIT'][0] if 'EBIT' in stock.financials else 0
    tax_rate = income_tax / ebit if ebit else 0.21  # Default to 21%

    # Get Risk-Free Rate and Market Return (estimated ~8%)
    risk_free_rate = get_risk_free_rate()
    market_return = 0.08

    #Calculate Cost of Equity (Re) using CAPM
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
    
    # Total Value (V = E + D)
    total_value = market_cap + total_debt

    # Calculate WACC
    wacc = (market_cap / total_value * cost_of_equity) + (total_debt / total_value * cost_of_debt * (1 - tax_rate))

    return round(wacc, 4) * 100
    
#past 4 year revenue growth
def get_revenue_growth(ticker):
    stock= yf.Ticker(ticker)
    revenue= stock.income_stmt.loc['Total Revenue'].dropna()
    revenue = revenue.sort_index()

    growth_rates = revenue.pct_change()
    avg_growth = pd.Series(growth_rates[1:]).mean()

    return avg_growth
    
#5 year revenue projection
def get_revenue_projection(ticker, growth_rate):
    stock= yf.Ticker(ticker)
    revenue_projection = []
    projection_years=[]
    last_year = stock.income_stmt.columns[0].year
    last_year_revenue= stock.income_stmt.loc['Total Revenue'].iloc[0]/1e9
    
    for x in range(5):
        next_year_revenue = (last_year_revenue * (1 + growth_rate))
        revenue_projection.append(round(next_year_revenue, 2))
        last_year_revenue = next_year_revenue
    
        projection_years.append(last_year + (x + 1))
        
    return pd.Series(revenue_projection, projection_years) 

def get_ebit_margin(ticker):
    stock= yf.Ticker(ticker)
    if 'Operating Income' in stock.income_stmt.index:
        operating_income = stock.income_stmt.loc['Operating Income'].dropna()
        revenue = stock.income_stmt.loc['Total Revenue'].dropna()
        operating_margin = operating_income/ revenue
        
        return operating_margin.mean()
        
    else:
        print("Operating Income not found")
def get_ebit_projection(revenue_projection, ebit_margin):
    ebit_projection = []
    
    for revenue in revenue_projection:
        ebit = revenue * ebit_margin
        ebit_projection.append(round(ebit, 2))
    return pd.Series(ebit_projection, revenue_projection.index) 

def get_NOPAT(ebit_projection):
    ebit_after_tax = []
    tax_rate= 0.21

    for ebit in ebit_projection:
        nopat = ebit * (1 - tax_rate)
        ebit_after_tax.append(nopat)
    return pd.Series(ebit_after_tax, ebit_projection.index) 
def get_depreciation_and_amortization(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Depreciation And Amortization' in stock.cashflow.index:
        da = stock.cashflow.loc['Depreciation And Amortization'].dropna()/1e9
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        da_rate = (da/revenue).mean()
        return da_rate * projection
    else:
        print("Depreciation & Amortization not found")

def get_capex(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Capital Expenditure' in stock.cashflow.index:
        capex = abs(stock.cashflow.loc['Capital Expenditure'].dropna()/1e9)
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        capex_rate = (capex/revenue).mean()
        return capex_rate * projection
    else:
        print("Capital Expenditure not found")

def get_workingcapital(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Change In Working Capital' in stock.cashflow.index:
        nwc = abs(stock.cashflow.loc['Change In Working Capital'].dropna()/1e9)
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        nwc_rate = (nwc/revenue).mean()
        return nwc_rate * projection
    else:
        print("Change In Working Capital not found")
def get_fcf(NOPAT, da, capex, nwc):
    fcf = NOPAT + da - capex - nwc
    return fcf

def get_fcf_pv(fcf, wacc):

    for t in range(1,6):
        fcf_pv = fcf

In [214]:
ticker = "AAPL"
print(f"WACC for {ticker}: {get_wacc(ticker)}%")
rate = get_revenue_growth(ticker)
rate

WACC for AAPL: 8.66%


0.02338440383792928

In [192]:
projection = get_revenue_projection(ticker=ticker, growth_rate= rate)

In [194]:
projection.index[0]

2025

In [196]:
ebit_margin= get_ebit_margin(ticker)

In [198]:
ebit = get_ebit_projection(projection, ebit_margin)

In [200]:
ebit

2025    121.46
2026    124.30
2027    127.20
2028    130.18
2029    133.22
dtype: float64

In [202]:
NOPAT = get_NOPAT(ebit)

In [250]:
da = stock.cashflow.loc['Depreciation And Amortization'].dropna()/1e9
revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
da_rate = (da/revenue).mean()
da_rate * projection

2025    11.838040
2026    12.114926
2027    12.398024
2028    12.687925
2029    12.984631
dtype: float64

In [280]:
da_estimite = get_depreciation_and_amortization(ticker, projection)

In [284]:
capex = get_capex(ticker, projection)

In [292]:
nwc= get_workingcapital(ticker, projection)

In [310]:
fcf = get_fcf(NOPAT, da_estimite, capex, nwc)
fcf

2025     92.467291
2026     94.629352
2027     96.836985
2028     99.105813
2029    101.420039
dtype: float64

In [320]:
for t in range(1,6):
    print(t)

1
2
3
4
5


In [312]:
df= pd.concat([projection, ebit, NOPAT, da_estimite, capex,nwc, fcf ],
              keys=['Revenue', 'EBIT','NOPAT (EBIT-TAX)','D&A(+)','CapEX(-)', 'Change in NWC(-)','Unlevered FCF'], axis=1).T

In [314]:
df

,2025,2026,2027,2028,2029
Revenue,400.180000,409.540000,419.110000,428.910000,438.940000
EBIT,121.460000,124.300000,127.200000,130.180000,133.220000
NOPAT (EBIT-TAX),95.953400,98.197000,100.488000,102.842200,105.243800
D&A(+),11.838040,12.114926,12.398024,12.687925,12.984631
CapEX(-),11.025795,11.283683,11.547356,11.817367,12.093714
Change in NWC(-),4.298355,4.398891,4.501683,4.606945,4.714678
Unlevered FCF,92.467291,94.629352,96.836985,99.105813,101.420039


In [84]:
df.loc[len(df)] = ebit_projection

In [206]:
stock= yf.Ticker(ticker)
stock.cashflow

,2024-09-30,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Free Cash Flow,108807000000.0,99584000000.0,111443000000.0,92953000000.0,NaN
Repurchase Of Capital Stock,-94949000000.0,-77550000000.0,-89402000000.0,-85971000000.0,NaN
Repayment Of Debt,-9958000000.0,-11151000000.0,-9543000000.0,-8750000000.0,NaN
Issuance Of Debt,0.0,5228000000.0,5465000000.0,20393000000.0,NaN
Issuance Of Capital Stock,NaN,NaN,NaN,1105000000.0,880000000.0
Capital Expenditure,-9447000000.0,-10959000000.0,-10708000000.0,-11085000000.0,NaN
Interest Paid Supplemental Data,NaN,3803000000.0,2865000000.0,2687000000.0,3002000000.0
Income Tax Paid Supplemental Data,26102000000.0,18679000000.0,19573000000.0,25385000000.0,NaN
End Cash Position,29943000000.0,30737000000.0,24977000000.0,35929000000.0,NaN
Beginning Cash Position,30737000000.0,24977000000.0,35929000000.0,39789000000.0,NaN


In [123]:
operating_income= stock.income_stmt.loc['Operating Income'].iloc[:4]
revenue = stock.income_stmt.loc['Total Revenue'].iloc[:4]
operaging_margin = operating_income/ revenue
operaging_margin.mean()

0.11544965128334249

In [125]:
operaging_margin = operating_income/ revenue
operaging_margin.mean()

0.11544965128334249

In [66]:
last_year_revenue= stock.income_stmt.loc['Total Revenue'].iloc[0]

In [429]:
last_year_revenue

97690000000.0

In [503]:
revenue_projection = []
projection_years=[]
avg_growth = get_revenue_growth(ticker)
last_year = stock.income_stmt.columns[0].year

for x in range(5):
    next_year_revenue = last_year_revenue * (1 + avg_growth)
    revenue_projection.append(round(next_year_revenue, 2))
    last_year_revenue = next_year_revenue

    projection_years.append(last_year + (x + 1))
    
pd.Series(revenue_projection, projection_years)    

2025    1.781799e+11
2026    1.823466e+11
2027    1.866106e+11
2028    1.909744e+11
2029    1.954402e+11
dtype: float64

In [499]:
revenue_projection

[158732108920.25,
 162443964657.29,
 166242619927.87,
 170130104487.34,
 174108495555.66]

In [479]:
last_year = stock.income_stmt.columns[0].year

In [481]:
projection_years

[2025, 2026, 2027, 2028, 2029]